# **Data Loading** 

In [59]:
import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torchvision import transforms
from torch.utils.data import DataLoader

from PIL import Image

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

dataset = ImageFolder("characters_train", transform=ToTensor())

In [60]:
train_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

Split the samples into train/val (80/20%) x image_path/lable sets:

In [61]:
paths = [s[0] for s in dataset.samples]
labels = [s[1] for s in dataset.samples]

train_paths, val_paths, train_labels, val_labels = train_test_split(
    paths,
    labels,
    test_size=0.2,
    stratify=labels,
    random_state=42
)

Since ImageFolder works only on directories, we create our own dataset class:

In [62]:
class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            img = self.transform(img)

        return img, label

Create data loaders:

In [63]:
train_dataset = CustomImageDataset(train_paths, train_labels, transform=train_transforms)
val_dataset   = CustomImageDataset(val_paths, val_labels, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=0)

# **Modeling**

Create the model class:

In [64]:
class CNNClassifier(nn.Module):
    def __init__(self, num_classes):
        super(CNNClassifier, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(16, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128*8*8, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

Create an epoch trainer:

In [65]:
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    all_preds = []
    all_labels = []

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(dataloader)
    macro_f1 = f1_score(all_labels, all_preds, average="macro")
    accuracy = (np.array(all_preds) == np.array(all_labels)).mean() * 100

    return avg_loss, accuracy, macro_f1


Create the epoch validator:

In [66]:
def validate_epoch(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(dataloader)
    macro_f1 = f1_score(all_labels, all_preds, average="macro")
    accuracy = (np.array(all_preds) == np.array(all_labels)).mean() * 100

    return avg_loss, accuracy, macro_f1


In [ ]:
num_classes = len(dataset.classes) 
model = CNNClassifier(num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 40
best_val_f1 = 0

for epoch in range(1, num_epochs + 1):
    train_loss, train_acc, train_f1 = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc, val_f1 = validate_epoch(model, val_loader, criterion, device)

    print(f"Epoch {epoch}/{num_epochs}")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | Train F1: {train_f1:.4f}")
    print(f"  Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.2f}% | Val   F1: {val_f1:.4f}")
    print("-" * 60)

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), "best_model.pth")

Epoch 1/40
  Train Loss: 2.7873 | Train Acc: 24.05% | Train F1: 0.0882
  Val   Loss: 2.0825 | Val   Acc: 44.59% | Val   F1: 0.1739
------------------------------------------------------------
Epoch 2/40
  Train Loss: 2.2033 | Train Acc: 38.57% | Train F1: 0.1519
  Val   Loss: 1.6781 | Val   Acc: 53.68% | Val   F1: 0.2215
------------------------------------------------------------
Epoch 3/40
  Train Loss: 1.9184 | Train Acc: 46.67% | Train F1: 0.1911
  Val   Loss: 1.5026 | Val   Acc: 56.90% | Val   F1: 0.2394
------------------------------------------------------------
Epoch 4/40
  Train Loss: 1.7239 | Train Acc: 51.25% | Train F1: 0.2171
  Val   Loss: 1.2944 | Val   Acc: 63.85% | Val   F1: 0.2844
------------------------------------------------------------
Epoch 5/40
  Train Loss: 1.5905 | Train Acc: 53.87% | Train F1: 0.2321
  Val   Loss: 1.2726 | Val   Acc: 63.64% | Val   F1: 0.2862
------------------------------------------------------------
Epoch 6/40
  Train Loss: 1.4746 | Train 